# CV for NMF
https://towardsdatascience.com/how-to-use-cross-validation-for-matrix-completion-2b14103d2c4c

In [1]:
import numpy as np

In [2]:
def cv_matrices(X, fold):
    """
    Given a matrix X, the function creates 4 sets of train + test matrices
    where each train matrix is masked with zeros in 0.25 of the values, and the
    test matrix is masked zeros in 0.75 of them.
    X - numpy array
    fold - is an integer from 0-3.
    Returns the masked data and also the masks for train and test
    """
    # Create a dict with the slicing indices
    rows = X.shape[0]
    cols = X.shape[1]
    mid_rows = int(rows/2)
    mid_cols = int(cols/2)
    
    idx_dict = {
                0: [[0,mid_rows],[0, mid_cols]],
                1: [[0,mid_rows],[mid_cols, cols]],
                2: [[mid_rows, rows], [0, mid_cols]],
                3: [[mid_rows, rows], [mid_cols, cols]]
    }
    
    idexes = idx_dict[fold]
    # Create masks
    train_mask = np.full((rows, cols), 1)
    train_mask[idexes[0][0]:idexes[0][1], idexes[1][0]:idexes[1][1]] = 0
    test_mask = 1 - train_mask
    
    
    # Create X_train
    X_train = X.copy()
    X_train[train_mask==0] = 0
    
    # Create X_test
    X_test = X.copy()
    X_test[train_mask==1] = 0
        
    return X_train, X_test, train_mask, test_mask

In [4]:
def nmf_cv(X, latent_features, cycles=5, max_iter=25000, curRes_test_prev_init=99999):
    """
    Decompose X to A*Y
    """
    eps = 1e-12
    # Main mask: take only the non-nan location for loss calculation. Later on (per fold) we combine
    # the fold mask into one mask
    mask = ~np.isnan(X)
    # Fill in nan to 0
    X = np.nan_to_num(X)
    
    # fold_tups is where we collect the attributes from each fold
    fold_tups = []
    # Cross validation by 4 folds
    for f in range(4):
        X_train, X_test, train_mask, test_mask = cv_matrices(X, f)
        
        # Creates train and test masks with nulls (combined mask) - after that we have zeros in the fold's
        # Quartile (test area) and also in the original nulls in the train area 
        train_null_mask = mask * train_mask
        test_null_mask = mask * test_mask
        
        # Takes several cycles per fold to get the minimum loss between them, due to the random initiation
        result_list = []
        for j in range(cycles): 
            # initial matrices. A is random and Y is A\X.
            rows, columns = X_train.shape
            # A is the user matrix. Now we initiate it randomly
            A = np.sum(np.nan_to_num(np.diag(X_train)))*np.random.randn(rows, latent_features)/np.sqrt(latent_features) 
            A = np.maximum(A, eps)

            # Y is the product matrix. We initiate it is a way that given A, we get the minimum Frobenius Form
            Y = linalg.lstsq(A, X_train)[0]
            Y = np.maximum(Y, eps)
            
            # Get only the train values
            masked_X = train_null_mask * X_train
            
            # Set test residual at a high value and train at 0
            curRes_test_prev = curRes_test_prev_init
            curRes_test_best = curRes_test_prev_init
            curRes_train_prev = 0
  
            train_list = [] # per cycle
            test_list = [] # per cycle
            
            # Initialize the resulted X (per cycle)
            X_est_prev = None
            X_est = np.dot(A, Y)
            
            # Takes several cycles per fold to get the minimum loss between them, due to the random initiation
        result_list = []
        for j in range(cycles): 
            # initial matrices. A is random and Y is A\X.
            rows, columns = X_train.shape
            # A is the user matrix. Now we initiate it randomly
            A = np.sum(np.nan_to_num(np.diag(X_train)))*np.random.randn(rows, latent_features)/np.sqrt(latent_features) 
            A = np.maximum(A, eps)

            # Y is the product matrix. We initiate it is a way that given A, we get the minimum Frobenius Form
            Y = linalg.lstsq(A, X_train)[0]
            Y = np.maximum(Y, eps)
            
            # Get only the train values
            masked_X = train_null_mask * X_train
            
            # Set test residual at a high value and train at 0
            curRes_test_prev = curRes_test_prev_init
            curRes_test_best = curRes_test_prev_init
            curRes_train_prev = 0
  
            train_list = [] # per cycle
            test_list = [] # per cycle
            
            # Initialize the resulted X (per cycle)
            X_est_prev = None
            X_est = np.dot(A, Y)
            
            # This for loop is the main algorithm procedure, updates the matrix till minimum residual or till break
            i = -1
            while X_est_prev is None or fit_residual > 1e-9:
                i += 1
                X_est_prev = X_est
                # Update A
                top = np.dot(masked_X, Y.T)
                bottom = (np.dot((train_null_mask * np.dot(A, Y)), Y.T)) + eps
                A *= top / bottom
                A = np.maximum(A, eps)
                # Update Y
                top = np.dot(A.T, masked_X)
                bottom = np.dot(A.T, train_null_mask * np.dot(A, Y)) + eps
                Y *= top / bottom
                Y = np.maximum(Y, eps)
                # New X matrix based on updated A and Y
                X_est = np.dot(A, Y)
                                     
                err = train_null_mask * (X_est_prev - X_est)
                fit_residual = np.sqrt(np.sum(err ** 2))
                if i > max_iter:
                    break
                    
                    # Collect results per cycle
            result_list.append((A, Y, curRes_train_prev, curRes_test_prev, i, train_list, test_list))

        result_list_sorted = list(sorted(result_list, key=lambda x: x[2]))
        # Get all the attributes of the best cycle per fold
        fold_best_curRes_test = result_list_sorted[0][3]
        fold_best_curRes_train = result_list_sorted[0][2]
        fold_A = result_list_sorted[0][0]
        fold_Y = result_list_sorted[0][1]
        fold_tup = (fold_A, fold_Y, fold_best_curRes_train, fold_best_curRes_test)
        fold_tups.append(fold_tup)
        
        
    # Get avg train/test score from all folds
    train_mean = np.mean([x[2] for x in fold_tups])
    test_mean = np.mean([x[3] for x in fold_tups])
    # Get all combinaitions of A and Y
    mean_A = [x[0] for x in fold_tups]
    mean_Y = [x[1] for x in fold_tups]
    
    return train_mean, test_mean, mean_A, mean_Y

In [ ]:
# Run multiple k and capture results

n_iter = 25000
k_tup_list = []
# k must be lower than the min of (columns/rows)
# k_max = min(orig_matrix_round_plus.shape[0], orig_matrix_round_plus.shape[1])
k_max = 400
X = np.memmap('/data/bioprotean/ABA/MEMMAP/genes_list/finalgenes_pos_L2.mymemmap',\
dtype='float32', mode='r', shape=(159326,2941))

for k in range(2,k_max):
    
    train_mean, test_mean, final_As, final_Ys = nmf_cv(X, k, max_iter=n_iter)
    k_tup = (train_mean, test_mean, final_As, final_Ys, k)
    k_tup_list.append(k_tup)

In [ ]:
# Extracting the best run
# Get the min test error and its index
test_err = [tup[1] for tup in k_tup_list]
min_test_err = np.min(test_err)
min_test_err_idx = test_err.index(min_test_err)
# Extract the run and its attributes
best_run = k_tup_list[min_test_err_idx]
best_k = min_test_err_idx + 2 